In [1]:
hepatitis_data_path = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/hepatitis/hepatitis.data'

In [2]:
names_hepatitis = ['Class','Age','Sex','Steroid','Antivirals','Fatigue','Malaise',
                   'Anorexia','Liver_Big','Liver_Firm',
                   'Spleen_Palpable','Spiders','Ascities','Varices',
                   'Bilirubin','ALK_Phosphate','SGOT','Albumin',
                   'Protime','Histology']

In [3]:
hepatitis = pd.read_csv(hepatitis_data_path,names=names_hepatitis,na_values='?')

In [4]:
hepatitis.head()

,Class,Age,Sex,Steroid,Antivirals,Fatigue,Malaise,Anorexia,Liver_Big,Liver_Firm,Spleen_Palpable,Spiders,Ascities,Varices,Bilirubin,ALK_Phosphate,SGOT,Albumin,Protime,Histology
0,2,30,2,1.0,2,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,85.0,18.0,4.0,NaN,1
1,2,50,1,1.0,2,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,0.9,135.0,42.0,3.5,NaN,1
2,2,78,1,2.0,2,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.7,96.0,32.0,4.0,NaN,1
3,2,31,1,NaN,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.7,46.0,52.0,4.0,80.0,1
4,2,34,1,2.0,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,NaN,200.0,4.0,NaN,1


In [6]:
hepatitis['Class'].value_counts()

2    123
1     32
Name: Class, dtype: int64

In [10]:
mice_impute = MICE(verbose=False)

In [11]:
hepatitis_complete = mice_impute.complete(X=hepatitis)

In [13]:
y = hepatitis_complete[:,0]

In [20]:
y = np.where(y==1,1,0)

In [14]:
X = hepatitis_complete[:,1:]

In [15]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [16]:
gaussNB = GaussianNB()

In [24]:
cross_val_score(estimator=gaussNB, X=X, y=y, scoring='roc_auc',n_jobs=-1, cv=5).mean()

0.7824246031746032

In [36]:
mushroom = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data',
                       na_values='?',header = None)

In [37]:
mushroom.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [38]:
y = np.where(mushroom.loc[:,0]=='p',1,0)

In [39]:
mushroom.dropna().shape

(5644, 23)

In [40]:
from sklearn.preprocessing import Imputer

In [32]:
cat_impute = Imputer(strategy='most_frequent')

In [41]:
mushroom.dtypes

0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
22    object
dtype: object

In [43]:
for col in range(22):
    mushroom.loc[:,col+1] = mushroom.loc[:,col+1].astype('category')

In [44]:
mushroom.dtypes

0       object
1     category
2     category
3     category
4     category
5     category
6     category
7     category
8     category
9     category
10    category
11    category
12    category
13    category
14    category
15    category
16    category
17    category
18    category
19    category
20    category
21    category
22    category
dtype: object

In [45]:
cat_impute.fit_transform(X = pd.get_dummies(mushroom.loc[:,1:]))

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
def impute_with_mode(X): # X is a DataFrame with all categorical values
    ncol = X.shape[1]
    for col in range(ncol):
        max_freq = X.iloc[:,col].value_counts().idxmax()
        X.iloc[:,col].fillna(value = max_freq, inplace=True)
    return X

In [53]:
mushroom.loc[:,1].value_counts().idxmax()

'x'

In [54]:
mushroom_1 = mushroom.copy()

In [56]:
mushroom_imputed = impute_with_mode(mushroom_1.loc[:,1:])

In [57]:
from sklearn.preprocessing import LabelEncoder

In [58]:
le = LabelEncoder()

In [61]:
mushroom_imputed_encoded = np.zeros(mushroom_imputed.shape)
for col in range(mushroom_imputed.shape[1]):
    mushroom_imputed_encoded[:,col] = le.fit_transform(mushroom_imputed.iloc[:,col])

In [62]:
mushroom_imputed_encoded

array([[5., 2., 4., ..., 2., 3., 5.],
       [5., 2., 9., ..., 3., 2., 1.],
       [0., 2., 8., ..., 3., 2., 3.],
       ...,
       [2., 2., 4., ..., 0., 1., 2.],
       [3., 3., 4., ..., 7., 4., 2.],
       [5., 2., 4., ..., 4., 1., 2.]])

In [63]:
mnNB = MultinomialNB()

In [65]:
cross_val_score(mnNB, X=mushroom_imputed_encoded, y = y, cv=10, 
                n_jobs=-1, scoring='accuracy').mean()

0.7787274614514198